# <center> <u>**Resnet 50 avec augmentation de données** </u></center>



# 1. Import des données


In [1]:
import os 
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from sklearn.datasets import load_files

2023-07-03 12:56:13.867312: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 12:56:13.896256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# main folder
path_to_folder = '../data_bees_detection/dataset_classification'

classes = os.listdir(os.path.join(path_to_folder, 'train'))
nb_classes = len(classes)  

# function to load a split of the dataset
def load_dataset(path):
    data = load_files(path)
    target_files = np.array(data['filenames'])
    target_labels = keras.utils.to_categorical(np.array(data['target']), nb_classes)
    return target_files, target_labels



In [3]:
# load the datasets
train_files, train_labels = load_dataset(os.path.join(path_to_folder, 'train'))
valid_files, valid_labels = load_dataset(os.path.join(path_to_folder, 'validation'))
test_files, test_labels = load_dataset(os.path.join(path_to_folder, 'test'))

Définition de l'augmentation et de la normalisation

In [4]:
# def function to augment the images
import cv2 
from PIL import Image
import albumentations as A
from albumentations import (Compose, Rotate, HorizontalFlip, VerticalFlip, Affine, RandomBrightnessContrast, ChannelShuffle)
import albumentations as A

AUGMENTATIONS = Compose([
    Rotate(limit=[0,100], p=0.5),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    Affine(shear=[-45, 45], p=0.5),
    RandomBrightnessContrast(p=0.5)
])


In [5]:
# Batch generator

from keras.preprocessing import image
from keras.applications.resnet import preprocess_input
from keras.utils import Sequence
from keras.utils import load_img, img_to_array



def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return preprocess_input(np.expand_dims(x, axis=0))

class BatchGenerator(Sequence):

    def __init__(self, paths, labels, batch_size, augmentations,normalize=None):
        self.paths = paths
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augmentations
        self.normalize = normalize

    def __len__(self):
        # nb of batches per epoch
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def load_img(self, img_path):

        # loads img 
        img = load_img(img_path, target_size=(224,224))
        x = img_to_array(img)
        
        # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
        x =  np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        return x

    def apply_augmentation(self, batch_imgs):
        # apply augmentation to a batch of images

        batch_imgs = batch_imgs.astype(np.uint8)
        

        for i in range(len(batch_imgs)):
            batch_imgs[i] = self.augment(image=batch_imgs[i])['image']

        return batch_imgs
    
    def normalize_batch(self, batch_imgs):
        # normalize a batch of images

        batch_imgs = batch_imgs.astype(np.float32)

        # normalize with imagenet stats
        mean = [125.3, 123.0, 113.9]
        std  = [63.0,  62.1,  66.7]

        for i in range(len(batch_imgs)):
            batch_imgs[i] -= mean
            batch_imgs[i] /= std

        return batch_imgs
        


    def __getitem__(self, idx):
        # get batch at position index

        batch_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_imgs = np.zeros((len(batch_paths), 224, 224, 3))
        for i, f in enumerate(batch_paths):
            batch_imgs[i] = self.load_img(f)

        if self.augment: 
            batch_imgs = self.apply_augmentation(batch_imgs)

        if self.normalize:
            batch_imgs = self.normalize_batch(batch_imgs)
            
        return batch_imgs, batch_labels




In [6]:
ds_train = BatchGenerator(train_files, train_labels, 16, AUGMENTATIONS,True)
ds_valid = BatchGenerator(valid_files, valid_labels, 16,  AUGMENTATIONS,True)
ds_test = BatchGenerator(test_files, test_labels, 16,  AUGMENTATIONS,True)

# 2. Choix de l'architecture du modèle, des paramètres et hyperparamètres

In [7]:
from keras.applications.resnet import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

# we take resnet50 as convolutional base with weights trained on imagenet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# we add the classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax')(x)

# we create the final model
model = Model(inputs=base_model.input, outputs=predictions)


2023-07-03 12:56:18.186791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 12:56:18.191695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-03 12:56:18.191850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
from keras.optimizers import Adam

opti = Adam(learning_rate=0.0001)
metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
model.compile(optimizer=opti, loss='categorical_crossentropy', metrics=metrics)


In [9]:
# add callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# checkpoint to save the best model
path_to_weights = '/datafiles/classification/saved_weights/benchmark_weights/2_resnet_augmentation.h5'
checkpoint = ModelCheckpoint(filepath=path_to_weights, monitor='val_categorical_accuracy', save_best_only=True, save_weights_only=False)

# early stopping
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=10)

# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.1, patience=5, min_lr=0.00001)

# 3. Entrainement du modèle

In [10]:
#history = model.fit(ds_train, epochs=100, validation_data=ds_valid, callbacks=[checkpoint, early_stopping, reduce_lr])

# 4. Test

In [12]:
path_test = '../data_bees_detection/dataset_classification/test'

test_files, test_labels = load_dataset(path_test)

ds_test = BatchGenerator(test_files, test_labels, 16, None)

# load the best model
model.load_weights('/datafiles/classification/saved_weights/benchmark_weights/2_resnet_augmentation.h5')

# evaluate the model
model.evaluate_generator(ds_test)

/tmp/ipykernel_24066/3983120818.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(ds_test)
2023-07-03 13:04:40.207370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


[2382.351806640625,
 0.010671130381524563,
 0.010671130381524563,
 0.010671130381524563]

: 